In [2]:
import glob
import pandas as pd

In [3]:
def modify_dataframe(df, subject_id, rel, pol, conv_num):
    """データフレームを整形する

    Args:
        df (pd.DataFrame): データフレーム
        subject_id (int): 被験者ID
        rel (str): 関係条件
        pol (str): 口調条件
    """
    # dfから"内容"列がNoneの行を削除
    df = df.dropna(subset=['内容'])
    # dfに新たに"添削"列を追加する．値は全て NaN でOK．
    df = df.copy()
    df['添削'] = None
    df['添削']= df['添削'].astype(str)

    # つづけて，"発話"列が"S"の行について，
    # "添削③"，"添削②", "添削①", "内容"の優先順位で，Nullでない最初の値を代入
    for i, row in df.iterrows():
        if row['発話'] == 'S':
            if not pd.isna(row[f'添削③']):
                df.loc[i, '添削'] = row[f'添削③']
            elif not pd.isna(row[f'添削②']):
                df.loc[i, '添削'] = row[f'添削②']
            elif not pd.isna(row[f'添削①']):
                df.loc[i, '添削'] = row[f'添削①']
            else:
                df.loc[i, '添削'] = row['内容']
    # dfに"subject_id", "rel", "pol"列を追加し，値を代入
    df['被験者ID'] = subject_id
    df['関係条件'] = rel
    df['口調条件'] = pol
    df['会話ID'] = f"{subject_id}_{rel}_{pol}_{conv_num}"
    # dfに"発話ID"列を追加し，通し番号（0が最初）をつける
    df['発話ID'] = range(len(df))
    # dfの0列目は削除する
    df = df.drop(df.columns[0], axis=1)
    # "発話"列の列名を"話者"にする
    df = df.rename(columns={'発話': '話者'})
    # 列の順番を以下のように変更する
    #   被験者ID, 関係条件, 口調条件, 発話ID, 話者, 内容, 添削, OK ?，添削①，添削②，添削③
    df = df[['被験者ID', '関係条件', '口調条件', '会話ID', '発話ID', '話者', '内容', '添削', 'OK ?', '添削①', '添削②', '添削③']]
    # df
    return df

In [4]:
def process_excel(filepath):
    """エクセルファイルを整形する

    Args:
        filepath (str): エクセルファイルのパス

    Returns:
        pd.DataFrame: 整形されたデータフレーム（2つ）
    """

    # filepathのbasenameは例えば "D-6_先輩-後輩_敬語.xlsx" となっている
    # このうち，D-6 を subject_id
    # 先輩-後輩 を rel
    # 敬語 を pol として取り出す
    subject_id, rel, pol = filepath.split('/')[-1].split('_')[:3]
    rel = rel[:2] + "-" + rel[3:]
    pol = pol.split('.')[0]
    # print(rel)

    # validation check
    assert subject_id[0] in ('A', 'B', 'C', 'D')
    assert subject_id[1] == '-'
    assert subject_id[2].isdigit()
    assert rel in ('先輩-後輩', '先生-生徒')
    assert pol in ('敬語', 'タメ口')
    
    df = pd.read_excel(filepath, index_col=None)
    idx = df[df['内容'] == '内容'].index[0]
    df1 = df.iloc[:idx]
    df2 = df.iloc[idx+1:]

    df1 = modify_dataframe(df1, subject_id, rel, pol, 1)
    df2 = modify_dataframe(df2, subject_id, rel, pol, 2)
    
    return df1, df2

In [5]:
filepaths = sorted(glob.glob('data/*/*.xlsx'))
dfs = []
for filepath in filepaths:
    # print(filepath)

    # データ不備により除外（訂正後解除）
    if "B-6_先生―生徒_タメ口.xlsx" in filepath:
        continue
    if "C-3_先生―生徒_タメ口.xlsx" in filepath:
        continue
    
    df1, df2 = process_excel(filepath)
    dfs.extend([df1, df2])

In [6]:
df_all = pd.concat(dfs, ignore_index=True)
df_all.to_excel('data_all.xlsx', index=False)

In [7]:
# 被験者IDが *-6 のデータを抽出
df_A6 = df_all[df_all['被験者ID'].str.endswith('-6')]
# df_A6 以外のデータを抽出
df_others = df_all[~df_all['被験者ID'].str.endswith('-6')]

In [10]:
df_A6.to_excel('valid.xlsx', index=False)
df_others.to_excel('train.xlsx', index=False)